<a href="https://colab.research.google.com/github/gkdivya/EVA/blob/main/4_BackpropagationAndArchitecturalBasics/TeamNotebooks/EVA4_Session_2_WithDropout.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from __future__ import print_function
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

In [47]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 16, 3, padding=1) #input -? OUtput? RF
        self.conv1_bn = nn.BatchNorm2d(16)

        self.conv2 = nn.Conv2d(16, 16, 3, padding=1)
        self.conv2_bn = nn.BatchNorm2d(16)
        
        self.pool1 = nn.MaxPool2d(2, 2)
        
        self.conv3 = nn.Conv2d(16, 32, 3, padding=1)
        self.conv3_bn = nn.BatchNorm2d(32)
        
        self.conv4 = nn.Conv2d(32, 16, 3, padding=1)
        self.conv4_bn = nn.BatchNorm2d(16)
        
        self.pool2 = nn.MaxPool2d(2, 2)
        
        self.conv5 = nn.Conv2d(16, 16, 3)
        self.conv5_bn = nn.BatchNorm2d(16)

        self.conv6 = nn.Conv2d(16, 16, 3)
        self.fc = nn.Linear(16*3*3, 10)

    def forward(self, x):
        x = F.dropout((self.pool1(F.relu(self.conv2_bn(self.conv2(F.relu(self.conv1_bn(self.conv1(x)))))))), 0.2)
        x = F.dropout((self.pool2(F.relu(self.conv4_bn(self.conv4(F.relu(self.conv3_bn(self.conv3(x)))))))), 0.2)
        x = self.conv6(F.relu(self.conv5_bn(self.conv5(x))))
        #print(x.shape)
        x = F.avg_pool2d(x, kernel_size=3, stride=1, padding=1)
        x = x.view(x.size(0), -1)
        #print(x.shape)
        x = self.fc(x)
        return F.log_softmax(x)

In [48]:
!pip install torchsummary
from torchsummary import summary
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
model = Net().to(device)
summary(model, input_size=(1, 28, 28))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 16, 28, 28]             160
       BatchNorm2d-2           [-1, 16, 28, 28]              32
            Conv2d-3           [-1, 16, 28, 28]           2,320
       BatchNorm2d-4           [-1, 16, 28, 28]              32
         MaxPool2d-5           [-1, 16, 14, 14]               0
            Conv2d-6           [-1, 32, 14, 14]           4,640
       BatchNorm2d-7           [-1, 32, 14, 14]              64
            Conv2d-8           [-1, 16, 14, 14]           4,624
       BatchNorm2d-9           [-1, 16, 14, 14]              32
        MaxPool2d-10             [-1, 16, 7, 7]               0
           Conv2d-11             [-1, 16, 5, 5]           2,320
      BatchNorm2d-12             [-1, 16, 5, 5]              32
           Conv2d-13             [-1, 16, 3, 3]           2,320
           Linear-14                   

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:35: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.


In [49]:


torch.manual_seed(1)
batch_size = 128

kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=True, download=True,
                    transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False, transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)


In [50]:
from tqdm import tqdm
def train(model, device, train_loader, optimizer, epoch):
    model.train()
    pbar = tqdm(train_loader)
    for batch_idx, (data, target) in enumerate(pbar):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        pbar.set_description(desc= f'loss={loss.item()} batch_id={batch_idx}')


def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

In [51]:

model = Net().to(device)
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)

for epoch in range(1, 10):
    train(model, device, train_loader, optimizer, epoch)
    test(model, device, test_loader)

  0%|          | 0/469 [00:00<?, ?it/s]


Test set: Average loss: 0.1049, Accuracy: 9672/10000 (97%)



  0%|          | 0/469 [00:00<?, ?it/s]


Test set: Average loss: 0.0585, Accuracy: 9812/10000 (98%)



  0%|          | 0/469 [00:00<?, ?it/s]

...


Test set: Average loss: 0.0389, Accuracy: 9873/10000 (99%)



  0%|          | 0/469 [00:00<?, ?it/s]


Test set: Average loss: 0.0430, Accuracy: 9848/10000 (98%)



loss=0.011841371655464172 batch_id=468: 100%|██████████| 469/469 [00:11<00:00, 39.46it/s]



Test set: Average loss: 0.0340, Accuracy: 9898/10000 (99%)

